1. Création database

In [1]:
# creation db  OK
import sqlite3

###------------------- VARIABLES -------------------###

nom_database = 'database\db_tout_roule.db'

table_principale = 'missions'
table_secondaire_1 = 'vehicules'
table_secondaire_2 = 'employes'

###--------------- CONNEXION DATABASE --------------###

connexion = sqlite3.connect(nom_database)
curseur = connexion.cursor()

###---------------- CREATION TABLES SECONDAIRES ------------------------------------------------###

# TABLES "vehicules"
creation_table_vehicules = f"""
    CREATE TABLE IF NOT EXISTS {table_secondaire_1}(
        immat_veh TEXT PRIMARY KEY,
        type_veh TEXT NOT NULL
    )    
"""
curseur.execute(creation_table_vehicules)

# TABLE "employes"
creation_table_employes = f"""
    CREATE TABLE IF NOT EXISTS {table_secondaire_2}(
        id_emp TEXT PRIMARY KEY,
        nom_emp TEXT NOT NULL,
        prenom_emp TEXT NOT NULL,
        genre_emp TEXT NOT NULL,
        fonction_emp TEXT NOT NULL
     )    
"""
curseur.execute(creation_table_employes)

###---------- CREATION TABLE PRINCIPALE ------------------------------###

creation_table_missions = f"""
    CREATE TABLE IF NOT EXISTS {table_principale}(
        id_mis INTEGER PRIMARY KEY AUTOINCREMENT,
        km_depart INTEGER NOT NULL,
        km_retour INTEGER NOT NULL,
        immat_mis TEXT NOT NULL,
        id_emp INTEGER NOT NULL,
        date_depart DATE NOT NULL,
        date_retour DATE NOT NULL,
        commentaires TEXT,
        FOREIGN KEY(immat_mis) REFERENCES {table_secondaire_1}(immat_veh),
        FOREIGN KEY(id_emp) REFERENCES {table_secondaire_2}(id_emp)
    )    
"""
curseur.execute(creation_table_missions)

###---------- TRANSMISSION ET DECONNEXION DB -------###

connexion.commit()
connexion.close()

2. Remplissage table vehicules
['immat_veh', type_veh']

In [2]:
# Remplissage table vehicules OK
import random, string
import sqlite3

nom_database = 'database\db_tout_roule.db'
table_secondaire_1 = 'vehicules'
compteur = 20

def generate_camion():
    camions = ['citerne', 'benne', 'frigorifique', 'fourgon']
    return random.choice(camions)

def generate_immatriculation():
    letters = random.choices(string.ascii_uppercase, k=2)
    digits = random.choices(string.digits, k=3)
    letters2 = random.choices(string.ascii_uppercase, k=2)
    license_plate = ''.join(letters) + '-' + ''.join(digits) + '-' + ''.join(letters2)
    return license_plate


conn = sqlite3.connect(nom_database)
cursor = conn.cursor()

# Boucle pour insérer les données dans la table
for _ in range(compteur):
    immatriculation = generate_immatriculation()
    type_vehicule = generate_camion()

    # Insérer les données dans la table
    cursor.execute(f"INSERT INTO {table_secondaire_1} (immat_veh, type_veh) VALUES (?, ?)",
                   (immatriculation, type_vehicule))

conn.commit()
conn.close()

3. Remplissage table employes

['id_emp' , 'nom_emp' , 'prenom_emp' , 'genre_emp' , 'fonction_emp']

In [3]:
#  Remplissage table employes OK
import random
import sqlite3
from faker import Faker
from unidecode import unidecode
from datetime import datetime, timedelta
from genderize import Genderize

nom_database = 'database\db_tout_roule.db'
table_secondaire_2 = 'employes'
fonction_emp = ['Chauffeur', 'Manager']

# Nombre de lignes à générer
compteur = 100

fake = Faker()

def generate_id_unique(cursor):
    while True:
        # Generate a random date between 2001-01-01 and 2019-12-31
        start_date = datetime(2001, 1, 1)
        end_date = datetime(2019, 12, 31)
        random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
        # Format the date as YYYYMMDD
        formatted_date = random_date.strftime('%Y%m%d')
        # Concatenate 'ID' with the formatted date
        unique_id = 'ID' + formatted_date

        # Check if the generated ID already exists in the table
        cursor.execute(f"SELECT COUNT(*) FROM employes WHERE id_emp = ?", (unique_id,))
        count = cursor.fetchone()[0]

        if count == 0:
            return unique_id


def generate_nom():
    nom_emp = fake.last_name()
    return unidecode(nom_emp)

def generate_prenom():
    prenom_emp = fake.first_name()
    return unidecode(prenom_emp)

# def generate_sexe(prenom):
#     # Utiliser la fonction get_gender() du module gender-guesser
#     genre_emp = gender_guesser.Detector().predict(prenom)
#     return genre_emp


def generate_sexe(prenom):
    # Use Genderize to predict the gender
    gender_info = Genderize().get([prenom])
    gender = gender_info[0]['gender']
    return gender

def generate_fonction():
    # Utiliser random.choices pour obtenir 5% de 'Manager'
    choix = random.choices(fonction_emp, weights=[85, 15], k=1)
    return choix[0]

# Connexion à la base de données
conn = sqlite3.connect(nom_database)
cursor = conn.cursor()

# Remplissage de la table "employes"
for ligne in range(compteur):
    matricule = generate_id_unique(cursor)
    nom = generate_nom()
    prenom = generate_prenom()
    sexe = generate_sexe(prenom)
    fonction = generate_fonction()

    cursor.execute(f"INSERT INTO {table_secondaire_2} (id_emp, nom_emp, prenom_emp, genre_emp, fonction_emp) VALUES (?, ?, ?, ?, ?)",
               (matricule, nom, prenom, sexe, fonction))

# Valider et fermer la connexion
conn.commit()
conn.close()

4. Remplissage table missions

['id_miss' , 'km_depart' , 'km_retour' , 'immat_mis' , 'date_depart', 'date_retour']

In [5]:
# Remplissage table missions OK

import sqlite3
from datetime import datetime, timedelta
import random

# Replace these values with your actual table names
table_principale = 'missions'
table_secondaire_1 = 'vehicules'
table_secondaire_2 = 'employes'

# Connect to the SQLite database
conn = sqlite3.connect('database\db_tout_roule.db')
curseur = conn.cursor()

# Function to generate a random date within a given range
def random_date(start_date, end_date):
    return start_date + timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds())))


# Function to generate and insert data into the missions table
def insert_data():
    # Get the current date
    current_date = datetime.now().date()

    # Dictionary to store the last km_retour for each immat_mis
    last_km_retour = {}

# Dictionary to store the last km_retour for each immat_mis
last_km_retour = {}

# Loop to insert data into the missions table
for _ in range(1000):  # Adjust the loop count based on your requirement
    # Auto-increment id_mis
    id_mis = None

    # Select immat_mis based on the specified rules
    if _ < 20:
        immat_mis = curseur.execute(f"SELECT immat_veh FROM {table_secondaire_1} LIMIT 1 OFFSET {_}").fetchone()[0]
    else:
        immat_mis = curseur.execute(f"SELECT immat_veh FROM {table_secondaire_1} ORDER BY RANDOM() LIMIT 1").fetchone()[0]

    # Select a random id_emp
    id_emp = curseur.execute(f"SELECT id_emp FROM {table_secondaire_2} WHERE fonction_emp = 'Chauffeur' ORDER BY RANDOM() LIMIT 1").fetchone()[0]

    # Generate random km_depart for the first value of each immat_mis
    if immat_mis not in last_km_retour:
        km_depart = random.randint(50, 5000)
        date_depart = random_date(datetime(2023, 1, 2).date(), datetime(2023, 1, 31).date())
        last_km_retour[immat_mis] = {'km_retour': km_depart, 'date_retour': date_depart, 'km_depart': km_depart}  # Initialize the structure
    else:
        # Generate date_depart > date_retour of the previous mission
        date_depart = last_km_retour[immat_mis]['date_retour'] + timedelta(days=random.randint(1, 5))
        km_depart = last_km_retour[immat_mis]['km_retour']
        last_km_retour[immat_mis] = {'km_retour': km_retour, 'date_retour': date_depart, 'km_depart': km_depart}  # Update the structure

    # Generate random km_retour
    km_retour = km_depart + random.randint(50, 500)

    # Generate random date_retour greater than date_depart and not on a Saturday or Sunday
    date_retour = None
    while not date_retour or date_retour.weekday() >= 5:  # 5 and 6 represent Saturday and Sunday
        date_retour = date_depart + timedelta(days=random.randint(0, 5))

    # Update km_retour for the current immat_mis
    last_km_retour[immat_mis]['km_retour'] = km_retour

    # case commentaire
    commentaires = " "

    # Insert data into the missions table
    curseur.execute(f"INSERT INTO {table_principale} (km_depart, km_retour, immat_mis, id_emp, date_depart, date_retour, commentaires) VALUES (?, ?, ?, ?, ?, ?, ?)",
                    (km_depart, km_retour, immat_mis, id_emp, str(date_depart), str(date_retour), commentaires))

# Commit changes
conn.commit()


# Call the function to insert data into the missions table
insert_data()

# Close the connection
conn.close()